In [ ]:
import pandas as pd

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests 
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
print('Folium installed')
print('Libraries imported.')

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url)

soup = BeautifulSoup(html, 'html.parser')
soup.title.text

In [ ]:
soup.table.tr.td

In [ ]:
#df = pd.DataFrame(soup.table)
lst = []
table = soup.find_all('table')[0]
rows = table.find_all('tr')
for tr in rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    lst.append(row)
    
df = pd.DataFrame(lst, columns=["Postcode", "Borough","Neighborhood"])

print("Count", df.count())

In [ ]:
df.head()

In [ ]:
df = df[df["Borough"] != "Not assigned" ]
print("Count", df.count())
df.head(10)


In [ ]:
df.dropna(inplace=True)
print("Count", df.count())

In [ ]:
df

In [ ]:
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df['Borough']
df

In [ ]:
dfgrouped = df.replace('Not assigned','').groupby(["Postcode","Borough" ],as_index=False).agg(', '.join)
dfgrouped

In [ ]:

'''
This is the first 10 elements of the list at Wikipedia today: December 11th, 2019. Note there is only one M5A Postcode in the list
Postcode	Borough	Neighbourhood
M1A	Not assigned	Not assigned
M2A	Not assigned	Not assigned
M3A	North York	Parkwoods
M4A	North York	Victoria Village
M5A	Downtown Toronto	Harbourfront
M6A	North York	Lawrence Heights
M6A	North York	Lawrence Manor
M7A	Queen's Park	Not assigned
M8A	Not assigned	Not assigned
M9A	Downtown Toronto	Queen's Park
'''
dfgrouped.loc[dfgrouped['Postcode'] == 'M5A' ]

In [ ]:
dfgrouped

##### To complete this assignment it was required to divide the work as follows: First retrieve the information in HTML, extract the table from the source code of the Wikipedia page, clean the fields, and then create a Panda dataframe. 
##### After that, I had to discard null and empty fields, as well as replace undefined neighborhood values. Since there are more neighborhoods that belong to the same postcode, we group them and concatenate the values (comma separated) neighborhoods. in the same field

In [ ]:
dfgrouped.shape

In [ ]:
dfgeo = pd.read_csv("https://cocl.us/Geospatial_data")
dfgeo.rename(columns={"Postal Code": "Postcode"}, inplace=True)
dfgeo.head()

In [ ]:
dffinal = pd.merge(dfgrouped, dfgeo, on="Postcode" )
dffinal

In [ ]:
CLIENT_ID = 'KUE4RZE5CCZ0KWCQB05TQV3QNH1A2HTVEDMPQA1QK2C3OFGI' # your Foursquare ID
CLIENT_SECRET = 'X3OAO0LG4GZG1KOKSAEZBBGB1ZPF4TGVU3GY4VQ3CZWOZTP0' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
#search_query = 'Italian'
radius = 500
#print(search_query + ' .... OK!')

In [ ]:
latitude = dffinal["Latitude"]
longitude = dffinal["Longitude" ]
print(latitude, longitude)


In [ ]:
lst_results = []
for index, row in dffinal.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
    results = requests.get(url).json()
    lst_results.append(results) 
    #print(url)
lst_results

In [ ]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

In [ ]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]


dataframe_filtered

In [ ]:
#Initial values
latitude = 43.703940
longitude = -79.593130

venues_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# add a red circle marker to represent the initial position
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Initial Value',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the venues as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup= label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map